In [77]:
from __future__ import print_function
import pandas as pd
from config import *
import datetime
import os
import sys
import getopt
import pickle
from tqdm import tqdm
import os.path
from env import *
from termcolor import colored, cprint
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request



def authenticate():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('sheet_token.pickle'):
        with open('sheet_token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'sheet_credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('sheet_token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    return service

def get_elevlista_with_personummer_as_index(service, ELEVLISTA_ID):
    """
    Get Infometor (elevlista) file in Drive and return it as a Dataframe
    """
    RANGE = 'elevlista!A1:D'

     # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=ELEVLISTA_ID,
                                range=RANGE).execute()
    values = result.get('values', [])

    if not values:
        print('No data found in elevlista.')
    else:
        error_list = []

        klasser = []
        personnummers = []
        names = []
        groups = []

        for i, row in enumerate(values):
            if i > 0:
                # Print columns A and E, which correspond to indices 0 and 4.
                try: # this will take care of eventually empty cells.
                    klass = row[0]
                    personnummer = str(row[3])
                    name = row[1]
                    group = row[2]

                    klasser.append(klass)
                    personnummers.append(personnummer)
                    names.append(name)
                    groups.append(group)
                    # emails.append(email)
                except Exception as e:
                    error_list.append(i+1)

        _dict = {
            'Klass': klasser,
            'Namn': names,
            "Grupper": groups,
            'Personnummer': personnummers,
        }
        df = pd.DataFrame.from_dict(_dict)
        df.set_index('Personnummer', inplace=True)
    return df, error_list

def get_groupimport(service, EXTENS_ID):
    """
    Get Infometor (elevlista) file in Drive and return it as a Dataframe
    """
    RANGE = 'extens!A1:E'

     # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=EXTENS_ID,
                                range=RANGE).execute()
    values = result.get('values', [])

    if not values:
        print('No data found in elevlista.')
    else:
        error_list = []

        personnummers = []
        names = []

        for i, row in enumerate(values):
            if i > 0:
                # Print columns A and E, which correspond to indices 0 and 4.
                try: # this will take care of eventually empty cells.
                    personnummer = str(row[1])
                    lastname = str(row[2])
                    firstname = str(row[3])
                    personnummers.append(personnummer)
                    names.append(lastname+", "+firstname)
                    
                except Exception as e:
                    error_list.append(str(row[1]))

        _dict = {
            'Personid': personnummers,
            'Namn': names
        }
        df = pd.DataFrame.from_dict(_dict)
        df.set_index('Personid', inplace=True)

    return df, error_list

def get_group_import_content(df_group, df_elev):
    """
    Se till att fylla upp grupplistan med rätt grupper efter vilket personid som finns.
    """
    content = []
    _errors = []
    for index_grp, row_grp in df_group.iterrows():
        
        # Tar bort bindestrecket från personid i df_group
        index_grp = index_grp.replace('-', '')
        if index_grp in df_elev.index:
            """
            Om personid finns i elevlistan
            """
            row = [index_grp, df_elev.loc[index_grp].Grupper, '']

        else:
            """
            För de personnumer som saknas kan det oftast vara så att det är de fyra
            sista som inte stämmer. Letar därför igenom elevlistan och se om det finns
            match både för de 6 första siffrorna i personnumret och för namnet.
            Avlägger sedan en rapport för detta till användaren så att man kan dubbelkolla
            att det blev rätt.
            """
            
            # De första sex siffrorna i personnummret i extens
            # som skall jämföras med de första sex siffrorna i
            # elevlistan nu när det inte matchade med alla tio 
            # siffrorna.
            pn_first_part_grp = index_grp[0:6]
            
            # En bool för att kunna fånga upp om en match helt saknas efter
            # att ha kollat första sex siffrorna och namnet.
            missing_match = True
            for index_elv, row_elv in df_elev.iterrows():
                # Tar bort bindestrecket från personid i df_elev.
                # Skall inte finnas men gör det fall i fall.
                index_elv = index_elv.replace('-', '')
                
                # De första sex siffrorna i elevlistan.
                pn_first_part_elv = index_elv[0:6]
                
                # kolla om det finns en rad där första sex siffrorna i extens och 
                # elevlistan stämmer överräns samtidigt som namnet är detsamma
                if pn_first_part_grp == pn_first_part_elv and row_grp.Namn == row_elv.Namn and missing_match:
                    row = [index_grp, df_elev.loc[index_elv].Grupper, df_elev.loc[index_elv].Namn +": "+index_elv+" ->  "+ row_grp.Namn +": "+index_grp] # lägg till personid, grupperna, namnet samt pn i elevlista -> pn i extens
                    missing_match = False
                    
            if missing_match:
                row = [index_grp, 'MISSING MATCH', 'MISSING MATCH']
                _errors.append(index_grp)
        
        content.append(row)
        
    
    return content, _errors

def clear_sheet(service, _range, ID):
    service.spreadsheets().values().clear(spreadsheetId=ID, range=_range).execute()
                
def edit_sheet(service, content, _range, ID):
    try:
        range = _range
        values = content
        resource = {
            "values": values
        }
        # use append to add rows and update to overwrite
        service.spreadsheets().values().update(spreadsheetId=ID, range=range, body=resource, valueInputOption="USER_ENTERED").execute()
    except Exception as e:
        print("While trying to append values error: ", e)
        
    
    
service = authenticate()
df_group, _errors_gruppimport = get_groupimport(service, EXTENS_ID)
df_elev, _errors_elevlista = get_elevlista_with_personummer_as_index(service, ELEVLISTA_ID)
content, _errors = get_group_import_content(df_group, df_elev)

clear_sheet(service, 'gruppimport!A2:C', ELEVLISTA_ID)
edit_sheet(service, content, "gruppimport!A2", ELEVLISTA_ID)

print(f'ERRORS: {_errors}')



ERRORS: []
